In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Embedding , Bidirectional , Dropout , LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

#load the dataset
vocab_size = 10000
max_length = 300 

(X_train , y_train) , (X_test , y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure the uniform input shape 
X_train = pad_sequences(X_train , maxlen = max_length , padding = 'post' , truncating = 'post')
X_test = pad_sequences(X_test , maxlen = max_length , padding = 'post' , truncating = 'post')

#Build the model 
model = Sequential([
    Embedding(  vocab_size , 128 , input_length=max_length),

    Bidirectional(LSTM(64 , return_sequences=False)) ,

    Dropout(0.3),
    Dense(64 , activation='relu'),
    Dropout(0.3),
    Dense(1 , activation='sigmoid')
])

#compile the model
model.compile(optimizer='adam' , 
             loss='binary_crossentropy',
             metrics=['accuracy'])

history = model.fit(X_train , y_train , 
                   epochs = 5 ,
                   batch_size=32 ,
                   validation_split=0.2 ,
                   verbose = 1)

# Evaluate the model
test_loss , test_accuracy = model.evaluate(X_test , y_test , verbose = 0 )
print(f"test Acuuracy : {test_accuracy * 100:.2f}%")

# Make predictions on test data
predictions = model.predict(X_test)

# Convert probabilities to binary (0 or 1)
predicted_labels = (predictions > 0.5).astype("int32")

# Print first 10 predictions and actual labels
for i in range(10):
    print(f"Review #{i+1}:")
    print(f"Predicted Label: {'Positive' if predicted_labels[i][0] == 1 else 'Negative'}")
    print(f"Actual Label: {'Positive' if y_test[i] == 1 else 'Negative'}")
    print("-" * 40)


# Visualize training history
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()
